In [2]:
#setup data
import csv
import pandas as pd
import numpy as np
from sklearn import preprocessing

#define risk free rate(rfr) as the three month treasure yield (DGS3MO)
rfr = np.array((5244,1))
#define term spread as 10 year treasure interest(DGS10) minus 1 year treasure interest(DGS1)
ts = np.array((5244,1))
#define default spread as the yield gap between AAA(DAAA) and BBB(DBAA) grade bonds
ds = np.array((5244,1))
ones = np.ones((5243,1))[:,0]

data1 = pd.read_csv('spread.csv')
data2 = pd.read_csv('StockMarket.csv')
data = pd.merge(data1, data2, on='Date')
rfr = data['DGS3MO']
ds = data['DGS10'] - data['DGS1']
ts = data['DBAA'] - data['DAAA']
# print(rfr.shape)
# print(ds.shape)
# print(ts.shape)
# print(ones.shape)
# A = np.stack((rfr, ds, ts, ones), axis = 1)

rfr2=np.zeros((5243,))
ds2=np.zeros((5243,))
ts2=np.zeros((5243,))
for i in range(5243):
    rfr2[i] = rfr[i+1]-rfr[i]
    ds2[i] = ds[i+1]-ds[i]
    ts2[i] = ts[i+1]-ts[i]
#delete the last data since there is no label for the next day
# print(data)
print(rfr[1:].shape)
print(rfr2.shape)
A = np.stack((rfr[1:], ds[1:], ts[1:], rfr2, ds2, ts2, ones), axis = 1)
A = preprocessing.normalize(A)
# print(A)

#the yield of three major index: Dow Jones, SP 500 and Nasdaq
B_raw = np.stack((data['DJIA'], data['SP500'], data['NASDAQ']), axis = 1)
#Labeled as -1 if the market loss the next day, +1 otherwise 
B = np.zeros((5244, 3))
for i in range(5244):
    for j in range(3):
        B[i][j] = -1 if B_raw[i][j] < 0 else 1

#delete first label since there is no previous day data to predict
B= np.delete(B, 0, 0)

(5243,)
(5243,)


In [3]:
#LASSO, this code is copied from previous assignment
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X


In [4]:
##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,525],[526,1050],[1051,1575],[1576,2100],[2101,2625],[2626,3150],[3151,3675],[3676,4200],[4201,4725],[4726,5243]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

lam_vals = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
er_sq = np.zeros((cases,1))
er_rate = np.zeros((cases,1))

In [5]:
# DJIA
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]
    
    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,0].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,0], 2) / len(Bv2[:,0])
    error_vec2 = [0 if Bv2[j][0]==Bp2[j] else 1 for j in range(len(Bv2[:,0]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,0])

[ 0.08036904  0.05986824 -0.0477641   2.43947407  9.90650101 -0.
  0.08890593]
[ 0.04779435  0.05841017 -0.14733096  4.17609316 10.91987034 -0.
  0.22801151]
[-1.41639770e-02  1.27278087e-04 -9.42230595e-02  2.93776971e+00
  9.45296114e+00 -0.00000000e+00  2.69581207e-01]
[-9.02092539e-03  1.46648727e-02  0.00000000e+00  0.00000000e+00
  1.13108383e+01 -0.00000000e+00  1.64663665e-01]
[-0.01623114 -0.08573469 -0.21891734  1.67038219 10.88614437  0.164961
  0.53274907]
[-0.         -0.01757203 -0.07230785  4.33011683  7.32350477 -0.
  0.2856456 ]
[ 0.02284861  0.06673981 -0.01632593  1.27013733  6.51539141 -0.
  0.        ]
[ 0.0267635   0.03359929 -0.0223031   1.00986665  7.41534624 -0.
  0.09534923]


In [6]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.05987137]
 [0.05852302]
 [0.05901689]
 [0.06023386]
 [0.05111013]
 [0.05237229]
 [0.05446616]
 [0.06236459]]
Average square error: 
 0.05724478803213348
Error rate for each case: 
 [[0.47047619]
 [0.44952381]
 [0.45714286]
 [0.47619048]
 [0.34285714]
 [0.36      ]
 [0.38416988]
 [0.51047619]]
Average error rate: 
 0.4313545688545689


In [7]:
# SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,1].reshape((Bt.shape[0],1)),lam_vals)
  
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,1], 2) / len(Bv2[:,1])
    error_vec2 = [0 if Bv2[j][1]==Bp2[j] else 1 for j in range(len(Bv2[:,1]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,1])

[ 0.10295775  0.09408032 -0.          0.          6.19963711 -0.
  0.        ]
[ 0.03541233  0.05092421 -0.10000606  4.36730054 10.67503204 -0.61221141
  0.23030008]
[-0.03202021 -0.         -0.          2.19386896  8.39756178 -0.
  0.19916111]
[-0.01227806  0.03054814  0.          0.          8.51924428 -0.
  0.15408294]
[-0.         -0.03840914 -0.          0.09423001  9.46025245 -0.
  0.2562892 ]
[-7.46634294e-05 -1.48330685e-02 -9.09843177e-03  3.77408274e+00
  6.23414398e+00 -0.00000000e+00  2.34450450e-01]
[ 0.          0.04164808 -0.          3.84573874  8.65084031 -0.
  0.10912329]
[ 0.01607843  0.02770232 -0.02679579  4.15103539  9.53928131 -0.
  0.15943008]


In [8]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06340311]
 [0.05789975]
 [0.06095238]
 [0.06083322]
 [0.05039526]
 [0.05306053]
 [0.05487518]
 [0.06259686]]
Average square error: 
 0.058002036013386035
Error rate for each case: 
 [[0.52761905]
 [0.44      ]
 [0.48761905]
 [0.48571429]
 [0.33333333]
 [0.36952381]
 [0.38996139]
 [0.51428571]]
Average error rate: 
 0.4435070785070785


In [9]:
# NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    Bv1 = B[v1_ind,:]
   
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    Bv2 = B[v2_ind,:]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    Bt = B[trn_ind,:]

    # Use training data to learn classifierA
    W = ista_solve_hot(At,Bt[:,2].reshape((Bt.shape[0],1)),lam_vals)
    #calculate error in the 26 w coresponding to each lambda
    er = np.zeros((26, 1))#error of the 26 lambda cases
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        Bp1 = np.sign(Av1@W[:,i])#predicted b
        error_vec1 = [0 if Bv1[j][0]==Bp1[j] else 1 for j in range(len(Bv1))]
        er[i] = sum(error_vec1) / len(Bv1)
        #select the best lambda with the least error rate
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[miniErIndex]
    #get best w from the lambda
    w_b = W[:,miniErIndex]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    Bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(Bp2 - Bv2[:,2], 2) / len(Bv2[:,2])
    error_vec2 = [0 if Bv2[j][2]==Bp2[j] else 1 for j in range(len(Bv2[:,2]))]
    er_rate[j] = sum(error_vec2) / len(Bv2[:,2])

[ 0.06648327  0.06850848 -0.          0.          7.10061309 -0.
  0.09739811]
[ 0.06710824  0.11767862 -0.22551848  3.52265516  9.48416035 -0.62017677
  0.28508892]
[-0.01037975  0.03237035 -0.09397654  2.30055291  8.15610979 -0.
  0.28681094]
[-0.          0.0321353  -0.          0.          8.75168376 -0.
  0.19422174]
[-0.01706789 -0.0751998  -0.33516526  1.61865668  8.7671602   0.04167634
  0.69485628]
[-0.02324134 -0.04252149 -0.14427361  3.16119691  6.21317359 -0.20836392
  0.49543587]
[ 2.01229616e-02  5.99876972e-02 -7.98089942e-03  2.84665801e+00
  8.02213126e+00 -0.00000000e+00  8.72630494e-02]
[-1.59834760e-02 -2.41513508e-03 -2.19637563e-01  3.56259953e+00
  8.93475105e+00 -8.51474685e-01  4.86517044e-01]


In [10]:
print("The classifier based on Nasdaq index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Nasdaq index:
Square error for each case: 
 [[0.06224813]
 [0.05852302]
 [0.05789975]
 [0.06071382]
 [0.05153428]
 [0.05400932]
 [0.05555017]
 [0.06047432]]
Average square error: 
 0.057619100911444746
Error rate for each case: 
 [[0.50857143]
 [0.44952381]
 [0.44      ]
 [0.48380952]
 [0.34857143]
 [0.38285714]
 [0.3996139 ]
 [0.48      ]]
Average error rate: 
 0.43661840411840414


In [14]:
#Ridge Regression
#Dow Jones
for j in range(cases):
        # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,0]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,0]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,0]
    

    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)


[ 0.08853657  0.0787489  -0.12009569  0.1689477   0.69758885 -0.02112105
  0.11993684]
[ 0.07184086  0.09848296 -0.15687952  0.18176761  0.64689437 -0.02299645
  0.1485909 ]
[-0.00572101  0.0330991  -0.15244322  0.22299614  0.68765842 -0.02166047
  0.26560843]
[-0.01478094  0.02602155 -0.05211053  0.05040909  0.74131176 -0.00157158
  0.19392365]
[ 0.0111633  -0.04268383 -0.04950572  0.0541161   0.58895615  0.01358938
  0.25997881]
[ 0.00790516 -0.01051843 -0.06496415  0.22236071  0.39506894 -0.00652432
  0.24555262]
[ 0.02724909  0.08047955 -0.13184656  0.22319353  0.55007252 -0.04335162
  0.09671964]
[ 0.03746187  0.05860072 -0.13584426  0.21548329  0.66055527 -0.03726577
  0.16014676]


In [15]:
print("The classifier based on Dow Jones index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on Dow Jones index:
Square error for each case: 
 [[0.06305884]
 [0.06047432]
 [0.05777429]
 [0.06213145]
 [0.05752255]
 [0.06011327]
 [0.05739795]
 [0.06142673]]
Average square error: 
 0.05998742395974471
Error rate for each case: 
 [[0.52190476]
 [0.48      ]
 [0.43809524]
 [0.50666667]
 [0.43428571]
 [0.47428571]
 [0.42664093]
 [0.4952381 ]]
Average error rate: 
 0.4721396396396397


In [16]:
#SP500
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,1]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,1]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,1]
    
    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

[ 0.10658396  0.10276352 -0.06551069  0.17096253  0.66887173 -0.01855277
  0.05378851]
[ 0.0601026   0.08858066 -0.06796867  0.17787478  0.62739111 -0.02209187
  0.1085875 ]
[-0.03577731  0.00586404 -0.06428932  0.22224824  0.6608169  -0.01933299
  0.25696362]
[-0.02763719  0.03173048 -0.05599671  0.04675769  0.6990628  -0.00259533
  0.21846683]
[ 1.43805913e-03 -3.03458567e-02 -4.08566732e-02  4.02661324e-02
  5.45510596e-01  1.77983163e-04  2.68632843e-01]
[ 0.00224777 -0.01045014 -0.02300527  0.20712558  0.35361783 -0.00660262
  0.22920385]
[ 0.0108661   0.06617536 -0.07474523  0.21746228  0.53408184 -0.04202059
  0.12721915]
[ 0.03108076  0.05976805 -0.09050426  0.21433792  0.62823689 -0.03714788
  0.14858446]


In [17]:
print("The classifier based on SP500 index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on SP500 index:
Square error for each case: 
 [[0.06419924]
 [0.05852302]
 [0.05852302]
 [0.06083322]
 [0.05752255]
 [0.06011327]
 [0.05817189]
 [0.06178009]]
Average square error: 
 0.05995828580623616
Error rate for each case: 
 [[0.54095238]
 [0.44952381]
 [0.44952381]
 [0.48571429]
 [0.43428571]
 [0.47428571]
 [0.43822394]
 [0.50095238]]
Average error rate: 
 0.4716827541827542


In [18]:
#NASDAQ
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(5243))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = A[v1_ind,:]
    bv1 = B[v1_ind,2]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = A[v2_ind,:]
    bv2 = B[v2_ind,2]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = A[trn_ind,:]
    bt = B[trn_ind,2]
    
    Wr = np.zeros((At.shape[1], 26))
    
    miniEr = 1
    miniErIndex = 0
    for i in range(26):
        #train the data with one of the each lambda and add into Wr
        Wr[:,i:i+1] = np.linalg.inv(At.T@At + lam_vals[i]*np.identity(At.shape[1]))@At.T@(bt.reshape(bt.shape[0],1))
        bp1 = np.sign(Av1@Wr[:,i])#predicted b
        error_vec1 = [0 if bv1[j]==bp1[j] else 1 for j in range(len(bv1))]
        er[i] = sum(error_vec1) / len(bv1)
        if miniEr > er[i]:
            miniEr = er[i]
            miniErIndex = i
            
    #get best lambda from above algorithm         
    lambda_b = lam_vals[i]
    #get best w from the lambda
    w_b = Wr[:,i]
    print(w_b)
    #calculate predicted label for the second holdout evaluation data
    bp2 = np.sign(Av2@w_b)
    #calculate the squared error and error rate
    er_sq[j] = np.linalg.norm(bp2 - bv2, 2) / len(bv2)
    error_vec2 = [0 if bv2[j]==bp2[j] else 1 for j in range(len(bv2))]
    er_rate[j] = sum(error_vec2) / len(bv2)

[ 0.07281958  0.08470219 -0.09279638  0.13086581  0.62342648 -0.02060694
  0.16042372]
[ 0.08692039  0.14263663 -0.14471496  0.1464252   0.57569901 -0.0233818
  0.13529384]
[-0.00176213  0.06262485 -0.13865532  0.17958439  0.60056109 -0.01778794
  0.27280907]
[-0.00287209  0.04596075 -0.07082379  0.05785924  0.63872374 -0.00213966
  0.23720442]
[ 0.02132333 -0.01902649 -0.07005554  0.05913912  0.50066963  0.00528522
  0.30016964]
[ 0.00968754 -0.00731664 -0.04600602  0.16617721  0.36207705 -0.0090287
  0.27860514]
[ 0.02858979  0.0795761  -0.0868529   0.17781872  0.51571316 -0.0321661
  0.12350818]
[ 0.01432264  0.04953983 -0.1178969   0.16015026  0.58589134 -0.03404797
  0.24436442]


In [19]:
print("The classifier based on NASDAQ index:")
print("Square error for each case: \n", er_sq)
print("Average square error: \n", np.mean(er_sq))
print("Error rate for each case: \n", er_rate)
print("Average error rate: \n", np.mean(er_rate))

The classifier based on NASDAQ index:
Square error for each case: 
 [[0.06305884]
 [0.06023386]
 [0.05739626]
 [0.06142673]
 [0.0563758 ]
 [0.05839889]
 [0.05829988]
 [0.05987137]]
Average square error: 
 0.05938270415575461
Error rate for each case: 
 [[0.52190476]
 [0.47619048]
 [0.43238095]
 [0.4952381 ]
 [0.41714286]
 [0.44761905]
 [0.44015444]
 [0.47047619]]
Average error rate: 
 0.4626383526383526
